In [1]:
import datetime
import pandas as pd
import datetime as dt

In [2]:
# TitreGenre : contient les informations sur les films (titre, genre, année de sortie, etc.)
# DescriptionEquipeFilm : contient les informations sur les membres de l'équipe de chaque film (IDfilm, IDpersonne, réalisateur, acteur, actrice)
# NomActeur : contient les informations sur les acteurs et actrices (nom et IDpersonne)
# TitreRegion : contient les informations sur les titres des films en fonction de la région (IDfilm, titre, région)
# NoteFilm : contient les informations sur les notes des films (IDfilm, note, nombre de votes)
# Imdb : contient les informations supplémentaires sur les films (IDfilm, backdrop_path, original_language, overview, poster_path, production_countries, video)

In [64]:
# Fichier principal où les filtres sont appliqués qui servira de base pour les merges avec les autres fichiers
# lecture du fichier title.basics.tsv.gz depuis le lien fourni et filtrage des données pour ne conserver que les films qui nous intéressent

TitreGenre = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t",low_memory=False)
TitreGenre = TitreGenre[TitreGenre['titleType'].isin(["movie"])]
TitreGenre['runtimeMinutes'] = pd.to_numeric(TitreGenre['runtimeMinutes'],errors='coerce')
TitreGenre = TitreGenre[(TitreGenre["runtimeMinutes"] > 40) & (TitreGenre["runtimeMinutes"] < 240)]
TitreGenre["startYear"] = pd.to_datetime(TitreGenre["startYear"], errors="coerce", format="%Y")
TitreGenre = TitreGenre[TitreGenre["startYear"] > pd.Timestamp("1945-01-01")]
TitreGenre["genres_list"] = TitreGenre["genres"].str.split(",")
TitreGenre = TitreGenre.drop("genres",axis=1)
TitreGenre = TitreGenre.drop("endYear",axis=1)
TitreGenre = TitreGenre[TitreGenre["isAdult"] == 0]
genres_exclus = {
    "Horror",
    "Music",
    "Sci-Fi",
    "Sport",
    "Musical",
    "News",
    "Adult",
    "Western",
    "Reality-TV",
    "Talk-Show",
    "Game-Show",
}

TitreGenre = TitreGenre[
    TitreGenre["genres_list"].apply(
        lambda genres:any(g not in genres_exclus for g in genres))
]

In [4]:
# Lecture du fichier title.akas.tsv.gz et sélection des colonnes pertinentes et filtrage pour ne conserver que les titres avec une région FR
TitreRegion = pd.read_csv("https://datasets.imdbws.com/title.akas.tsv.gz",
                           sep="\t", na_values="\\N",low_memory=False, usecols=["titleId","region"])
TitreRegion = TitreRegion[TitreRegion["region"] == "FR"]

In [5]:
# Lecture du fichier title.principals.tsv.gz et sélection des colonnes pertinentes pour merge
NomActeur = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz",
                        na_values="\\N", sep="\t", low_memory=False, usecols=["nconst", "primaryName"])

In [6]:
# Lecture du fichier title.principals.tsv.gz et sélection des colonnes pertinentes pour merge 
# et filtrage pour ne conserver que les réalisateurs, acteurs et actrices
DescriptionEquipeFilm = pd.read_csv(
        "https://datasets.imdbws.com/title.principals.tsv.gz",
        sep="\t",
        na_values="\\N",
        usecols=["tconst", "ordering", "nconst", "category"],
        low_memory=False
    )
DescriptionEquipeFilm = DescriptionEquipeFilm.loc[DescriptionEquipeFilm["category"].isin(["director", "actor", "actress"])]

In [42]:
# Lecture du fichier title.ratings.tsv.gz et sélection des colonnes pertinentes pour merge et filtrage pour ne conserver que les films ayant plus de 1000 votes et une note moyenne supérieure ou égale à 6.0
NoteFilm = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t",na_values="\\N", low_memory=False)
NoteFilm = NoteFilm[NoteFilm["numVotes"] > 100]
NoteFilm = NoteFilm[NoteFilm["averageRating"] >= 4.1]

In [63]:
# Lecture du fichier tmdb_full.csv et sélection des colonnes pertinentes pour merge et filtrage pour ne conserver que les films en anglais, français, espagnol, allemand, italien, japonais, portugais, néerlandais, norvégien, danois ou coréen
Imdb = pd.read_csv(r"C:\Users\anton\Desktop\Projet_2\Base_de_donnée\tmdb_full.csv", low_memory=False, usecols=["imdb_id", "backdrop_path", "original_language", "overview","poster_path", "production_countries","video"])
Imdb= Imdb[Imdb["original_language"].isin(["en","fr","es","de","it","pt","nl","no","da"])]

In [19]:
# Merge entre DescriptionEquipeFilm et NomActeur pour obtenir les noms des acteurs et actrices associés à chaque film
MergeDescriptionEquipeFilmNomActeur = pd.merge(
    DescriptionEquipeFilm,
    NomActeur,
    left_on="nconst",
    right_on="nconst",
    how="left"
)

In [20]:
# Groupement par film pour obtenir une liste des acteurs et actrices associés à chaque film
MergeDescriptionEquipeFilmNomActeur = MergeDescriptionEquipeFilmNomActeur.groupby("tconst").agg(
    {"category": lambda x: list(x), "primaryName": lambda x: list(x)}
).reset_index()

In [21]:
# merge entre MergeDescriptionEquipeFilmNomActeur et TitreGenre pour obtenir les informations sur les films associés à chaque acteur et actrice
MergeDescriptionEquipeFilmNomActeurTitreGenre = pd.merge(
    MergeDescriptionEquipeFilmNomActeur,
    TitreGenre,
    left_on="tconst",  
    right_on="tconst",
    how="inner"
)

In [22]:
# merge entre MergeDescriptionEquipeFilmNomActeurTitreGenre et TitreRegion pour obtenir les titres des films en fonction de la région
MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegion = pd.merge(
    MergeDescriptionEquipeFilmNomActeurTitreGenre,
    TitreRegion,
    left_on="tconst",
    right_on="titleId",
    how="inner"
)

In [43]:
# merge entre MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegion et NoteFilm pour obtenir les notes des films
MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegionNote = pd.merge(
    MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegion,
    NoteFilm,
    left_on="tconst",
    right_on="tconst",
    how="inner"
)

In [ ]:
#  merge entre MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegionNote et Imdb pour obtenir les informations supplémentaires sur les films (backdrop_path, original_language, overview, poster_path, production_countries, video)
FinalMerge = pd.merge(
    MergeDescriptionEquipeFilmNomActeurTitreGenreTitreRegionNote,
    Imdb,
    left_on="tconst",
    right_on="imdb_id",
    how="inner"
)
#Suppressions des doublons et filtrage pour ne conserver que les films en français ou les films non français sortis après 1990
FinalMerge = FinalMerge.drop(["titleType","isAdult","titleId","category"], axis=1)
FinalMerge = FinalMerge[(FinalMerge["original_language"] == "fr") |
                        ((FinalMerge["original_language"] != "fr") & (FinalMerge["startYear"] >= pd.Timestamp("2000-01-01")))]

In [59]:
FinalMerge["original_language"].value_counts()

original_language
en    25084
fr     7485
it     3091
de     1895
es     1796
Name: count, dtype: int64

In [62]:
FinalMerge["original_language"].value_counts(normalize=True)

original_language
en    0.637443
fr    0.190211
it    0.078549
de    0.048156
es    0.045641
Name: proportion, dtype: float64

In [51]:
FinalMerge["original_language"].value_counts()

original_language
en    7648
fr    2363
ja     668
es     558
de     372
ko     320
it     300
pt     134
da     115
nl     104
no      95
Name: count, dtype: int64

In [52]:
FinalMerge["original_language"].value_counts(normalize=True)

original_language
en    0.603297
fr    0.186401
ja    0.052694
es    0.044017
de    0.029344
ko    0.025243
it    0.023665
pt    0.010570
da    0.009072
nl    0.008204
no    0.007494
Name: proportion, dtype: float64

In [46]:
FinalMerge[FinalMerge["originalTitle"].str.contains("bronzés")]

,tconst,primaryName,primaryTitle,originalTitle,startYear,runtimeMinutes,genres_list,region,averageRating,numVotes,backdrop_path,imdb_id,original_language,overview,poster_path,production_countries,video
9916,tt0077276,"[Josiane Balasko, Luis Rego, Marie-Anne Chazel...",French Fried Vacation,Les bronzés,1978-01-01,105.0,"[Comedy, Romance]",FR,6.7,8443,/vFM5xph7dRZF86KoFocbUNtIQZ0.jpg,tt0077276,fr,Holidaymakers arriving in a Club Med camp on t...,/gjmvQEQsWqM6884raalqMYv2Smt.jpg,['FR'],False
10245,tt0078907,"[Josiane Balasko, Michel Blanc, Marie-Anne Cha...",French Fried Vacation 2,Les bronzés font du ski,1979-01-01,90.0,[Comedy],FR,7.4,10050,/8xovwlLYPqKaHwjhfrMcGJU9TcX.jpg,tt0078907,fr,In this sequel to Les Bronzes (1978) summer ha...,/uTN0bp8yMUKesNTLXR1GnL2kLFs.jpg,['FR'],False
24591,tt0445800,"[Josiane Balasko, Michel Blanc, Marie-Anne Cha...",French Fried Vacation 3: Friends Forever,Les bronzés 3: amis pour la vie,2006-01-01,97.0,[Comedy],FR,4.1,3870,/ic44gb1sfOvfQ4v1esFSjJQsASP.jpg,tt0445800,fr,"After the Club Med and skiing, what happened t...",/9g7z9WG6MS3Q8cYfxTTBC4Ckm9A.jpg,['FR'],False
